## 4.8 Grouping Data & Aggregating Variables

### Contents
01 Import Libraries and Data

02 Grouping and Aggregating Data

03 Performing Multiple Aggregations

04 Aggregating Data with transform()

05 Deriving Columns with loc()

06 Identify and Address Outlier Prices

07 Data Type Reduction for Memory Savings

08 Export Data to Pickle

### 01 Import Libraries and Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create path variable for main project folder
path = r'D:\JupyterProjects\06-2022 Instacart Basket Analysis'

In [3]:
# Import pickle file as ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged.pkl'))

In [4]:
# Check shape of dataframe
ords_prods_merge.shape

(32404859, 19)

In [5]:
# Create subset of one million rows
df_small = ords_prods_merge[:1000000]

In [6]:
# Check shape of subset dataframe
df_small.shape

(1000000, 19)

In [7]:
# Check head of subset dataframe
df_small.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders


### 02 Grouping and Aggregating Data

In [8]:
# Try groupby() function
df_small.groupby('product_name')

In [9]:
# Find the average number of orders by department in data subset
df_small.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [10]:
# Get same results using mean() function instead of agg()
df_small.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

### 03 Performing Multiple Aggregations

In [11]:
# Multiple aggregations on order_number column grouped by department_id
df_small.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

### 04 Aggregating Data with transform()

In [12]:
# Create max_order column applying transform () function to order_number grouped by user_id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [13]:
# Check head 15 rows
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10


### 05 Deriving Columns with loc()

In [14]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [15]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [16]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [17]:
# Check loyalty_flag frequencies
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [18]:
# View top 10 rows of specific columns
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(10)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


### 06 Identify and Address Outlier Prices

In [19]:
# Locate grocery item prices > $100
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
10030345,912404,17,12,2,14,5.0,False,21553,5,0,...,108,16,14900.0,both,High-range product,Regularly busy,Regularly busy,Average orders,40,Regular customer
10030346,603376,17,22,6,16,4.0,False,21553,3,1,...,108,16,14900.0,both,High-range product,Regularly busy,Regularly busy,Average orders,40,Regular customer
10030347,3264360,135,2,2,21,13.0,False,21553,6,0,...,108,16,14900.0,both,High-range product,Regularly busy,Regularly busy,Average orders,4,New customer
10030348,892534,135,3,0,8,12.0,False,21553,3,1,...,108,16,14900.0,both,High-range product,Busiest day,Busiest days,Average orders,4,New customer
10030349,229704,342,8,1,19,30.0,False,21553,9,0,...,108,16,14900.0,both,High-range product,Regularly busy,Busiest days,Average orders,16,Regular customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29166209,2249946,204099,29,0,8,4.0,False,33664,1,0,...,84,16,99999.0,both,High-range product,Busiest day,Busiest days,Average orders,39,Regular customer
29166210,2363282,204099,31,0,9,2.0,False,33664,1,1,...,84,16,99999.0,both,High-range product,Busiest day,Busiest days,Average orders,39,Regular customer
29166211,3181945,204395,13,3,15,8.0,False,33664,25,0,...,84,16,99999.0,both,High-range product,Regularly busy,Slowest days,Average orders,15,Regular customer
29166212,2486215,205227,7,3,20,4.0,False,33664,8,0,...,84,16,99999.0,both,High-range product,Regularly busy,Slowest days,Average orders,12,Regular customer


In [20]:
# Turn outlier prices into NaNs
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [21]:
# Check max price (price should not exceed 100)
ords_prods_merge['prices'].max()

25.0

### 07 Data Type Reduction for Memory Savings

In [22]:
ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
new_customer                  bool
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
dtype: object

In [23]:
# Simplify dataframe name
df = ords_prods_merge

In [24]:
# Check maximum value of field
df['order_id'].max()

3421083

In [25]:
# Reduce data type to accommodate maximum
df = df.astype({'order_id': 'int32'})

In [26]:
# Check maximum value of field
df['user_id'].max()

206209

In [27]:
# Reduce data type to accommodate maximum
df = df.astype({'user_id': 'int32'})

In [28]:
# Check maximum value of field
df['orders_day_of_week'].max()

6

In [29]:
# Reduce data type to accommodate maximum
df = df.astype({'orders_day_of_week': 'int8'})

In [30]:
df['order_hour_of_day'].max()

23

In [31]:
df = df.astype({'order_hour_of_day': 'int8'})

In [32]:
df['order_number'].max()

99

In [33]:
df = df.astype({'order_number': 'int8'})

In [34]:
# Check maximum value of field
df['department_id'].max()

21

In [35]:
# Reduce data type to accommodate maximum
df = df.astype({'department_id': 'int8'})

In [36]:
# Check maximum value of field
df['days_since_prior_order'].max()

30.0

In [37]:
# Reduce data type to accommodate maximum
df = df.astype({'days_since_prior_order': 'float16'})

In [38]:
# Check maximum value of field
df['product_id'].max()

49688

In [39]:
# Reduce data type to accommodate maximum
df = df.astype({'product_id': 'int32'})

In [40]:
# Check maximum value of field
df['add_to_cart_order'].max()

145

In [41]:
# Reduce data type to accommodate maximum
df = df.astype({'add_to_cart_order': 'int16'})

In [42]:
# Check maximum value of field
df['reordered'].max()

1

In [43]:
# Reduce data type to accommodate maximum
df = df.astype({'reordered': 'int8'})

In [44]:
# Check maximum value of field
df['aisle_id'].max()

134

In [45]:
# Reduce data type to accommodate maximum
df = df.astype({'aisle_id': 'int16'})

In [46]:
# Check maximum value of field
df['aisle_id'].max()

134

In [47]:
# Reduce data type to accommodate maximum
df = df.astype({'aisle_id': 'int16'})

In [48]:
# Check maximum value of field
df['prices'].max()

25.0

In [49]:
# Reduce data type to accommodate maximum
df = df.astype({'prices': 'float16'})

In [50]:
# Check maximum value of field
df['max_order'].max()

99

In [51]:
# Reduce data type to accommodate maximum
df = df.astype({'max_order': 'int16'})

In [52]:
df.dtypes

order_id                     int32
user_id                      int32
order_number                  int8
orders_day_of_week            int8
order_hour_of_day             int8
days_since_prior_order     float16
new_customer                  bool
product_id                   int32
add_to_cart_order            int16
reordered                     int8
product_name                object
aisle_id                     int16
department_id                 int8
prices                     float16
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int16
loyalty_flag                object
dtype: object

### 07 Export Data to Pickle

In [53]:
# Export ords_prods_merge
df.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'Four8NanData.pkl'))